# to_video Examples

This notebook generates example videos and thumbnail images for documentation.

Output images are saved to `docs/_images/to_video/` for use in rendered documentation.

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

import mbo_utilities as mbo
from mbo_utilities import to_video

# output directories
DOCS_IMAGES = Path("../_images/to_video")
DOCS_IMAGES.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {DOCS_IMAGES.resolve()}")

## Generate Synthetic Calcium Imaging Data

Create realistic-looking calcium imaging data with simulated cells and activity.

In [ ]:
def generate_calcium_data(n_frames=200, height=256, width=256, n_cells=15, seed=42):
    """Generate synthetic calcium imaging data with active cells."""
    rng = np.random.RandomState(seed)
    
    # base background with some texture
    data = rng.normal(500, 50, (n_frames, height, width)).astype(np.float32)
    
    # add cells with calcium transients
    for i in range(n_cells):
        # cell position and size
        cy, cx = rng.randint(30, height-30), rng.randint(30, width-30)
        radius = rng.randint(8, 15)
        
        # create cell mask (gaussian blob)
        yy, xx = np.ogrid[:height, :width]
        dist = np.sqrt((yy - cy)**2 + (xx - cx)**2)
        mask = np.exp(-dist**2 / (2 * radius**2))
        
        # generate calcium transients (random spikes)
        n_spikes = rng.randint(3, 8)
        spike_times = rng.choice(n_frames-20, n_spikes, replace=False)
        
        signal = np.zeros(n_frames)
        for t in spike_times:
            # fast rise, slow decay
            amp = rng.uniform(300, 800)
            tau = rng.uniform(10, 30)
            for dt in range(min(50, n_frames - t)):
                signal[t + dt] += amp * np.exp(-dt / tau)
        
        # add signal to data
        for t in range(n_frames):
            data[t] += signal[t] * mask
    
    # add some noise and clip
    data += rng.normal(0, 30, data.shape)
    data = np.clip(data, 0, 4095).astype(np.int16)
    
    return data

# generate data
data_3d = generate_calcium_data(n_frames=300, n_cells=20)
print(f"Generated data shape: {data_3d.shape}, dtype: {data_3d.dtype}")
print(f"Value range: [{data_3d.min()}, {data_3d.max()}]")

## Save Sample Frames for Documentation

In [ ]:
def save_frame_comparison(data, title, filename, vmin=None, vmax=None, gamma=1.0):
    """Save a single frame with processing applied."""
    frame = data[100].astype(np.float32)  # middle frame
    
    if vmin is None:
        vmin = np.percentile(frame, 1)
    if vmax is None:
        vmax = np.percentile(frame, 99.5)
    
    # normalize and apply gamma
    frame_norm = np.clip((frame - vmin) / (vmax - vmin), 0, 1)
    if gamma != 1.0:
        frame_norm = np.power(frame_norm, gamma)
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(frame_norm, cmap='gray', vmin=0, vmax=1)
    ax.set_title(title, fontsize=12)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(DOCS_IMAGES / filename, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"Saved: {filename}")

# save comparison frames
save_frame_comparison(data_3d, "Raw Frame", "frame_raw.png")
save_frame_comparison(data_3d, "Gamma 0.7 (Brighter)", "frame_gamma_07.png", gamma=0.7)
save_frame_comparison(data_3d, "Gamma 1.3 (Darker)", "frame_gamma_13.png", gamma=1.3)

## Example 1: Basic Export

In [ ]:
# basic export - default settings
output = to_video(
    data_3d,
    DOCS_IMAGES / "example_basic.mp4",
    fps=30,
    max_frames=100,
)
print(f"Created: {output}")

## Example 2: Speed Factor (10x Faster Playback)

In [ ]:
# 10x speed for quick preview
output = to_video(
    data_3d,
    DOCS_IMAGES / "example_10x_speed.mp4",
    fps=30,
    speed_factor=10,
    max_frames=200,
)
print(f"Created: {output}")

## Example 3: Enhanced Quality (Temporal Smoothing + Gamma)

In [ ]:
# high quality with smoothing and gamma correction
output = to_video(
    data_3d,
    DOCS_IMAGES / "example_enhanced.mp4",
    fps=30,
    speed_factor=5,
    temporal_smooth=3,
    gamma=0.8,
    quality=10,
    max_frames=200,
)
print(f"Created: {output}")

## Example 4: With Colormap

In [ ]:
# using viridis colormap
output = to_video(
    data_3d,
    DOCS_IMAGES / "example_viridis.mp4",
    fps=30,
    speed_factor=5,
    cmap="viridis",
    gamma=0.8,
    max_frames=150,
)
print(f"Created: {output}")

# save colormap frame for docs
frame = data_3d[100].astype(np.float32)
vmin, vmax = np.percentile(frame, [1, 99.5])
frame_norm = np.clip((frame - vmin) / (vmax - vmin), 0, 1)
frame_norm = np.power(frame_norm, 0.8)

fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(frame_norm, cmap='viridis')
ax.set_title("Viridis Colormap", fontsize=12)
ax.axis('off')
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(DOCS_IMAGES / "frame_viridis.png", dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print("Saved: frame_viridis.png")

## Example 5: 4D Data (Multi-plane)

In [ ]:
# create 4D data (T, Z, Y, X)
data_4d = np.stack([
    generate_calcium_data(n_frames=150, n_cells=15, seed=i)
    for i in range(3)  # 3 z-planes
], axis=1)
print(f"4D data shape: {data_4d.shape}")

# export specific z-plane
output = to_video(
    data_4d,
    DOCS_IMAGES / "example_4d_plane1.mp4",
    fps=30,
    speed_factor=5,
    plane=1,  # middle plane (0-indexed)
    gamma=0.8,
    max_frames=100,
)
print(f"Created: {output}")

## Summary: Output Files

In [ ]:
print("Generated files:")
print("=" * 50)
for f in sorted(DOCS_IMAGES.glob("*")):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:<30} {size_kb:>8.1f} KB")